#通过langchain的自定义LLM封装Doubao模型

自定义LLM，参考 https://blog.csdn.net/qq_39749966/article/details/136108973

豆包模型调用参考：
```
'''
Usage:
Ark v3 sdk
pip install 'volcengine-python-sdk[ark]'
'''

from volcenginesdkarkruntime import Ark

# fetch ak&sk from environmental variables "VOLC_ACCESSKEY", "VOLC_SECRETKEY"
# or specify ak&sk by Ark(ak="${YOUR_AK}", sk="${YOUR_SK}").
# you can get ak&sk follow this document(https://www.volcengine.com/docs/6291/65568)
client = Ark()

if __name__ == "__main__":
    # Non-streaming:
    print("----- standard request -----")
    completion = client.chat.completions.create(
        model="ep-20240531085255-6tmx9",
        messages=[
            {
                "role": "user",
                "content": "Say this is a test",
            },
        ]
    )
    print(completion.choices[0].message.content)

    # Streaming:
    print("----- streaming request -----")
    stream = client.chat.completions.create(
        model="ep-20240531085255-6tmx9",
        messages=[
            {
                "role": "user",
                "content": "How do I output all files in a directory using Python?",
            },
        ],
        stream=True
    )
    for chunk in stream:
        if not chunk.choices:
            continue

        print(chunk.choices[0].delta.content, end="")
    print()
```

需要安装的库
```
pip install langchain
pip install volcengine-python-sdk
```

In [11]:
from volcenginesdkarkruntime import Ark
from langchain.llms.base import LLM
from langchain import PromptTemplate, LLMChain
import logging
from typing import Any

class DoubaoLLM(LLM):

    model: str = "ep-20240531085255-6tmx9"
    client: Ark = None

    def __init__(self, akstr: str, skstr: str):
        super().__init__()
        self.client = Ark(ak=akstr, sk=skstr)
        
    @property
    def _llm_type(self) -> str:
        return "Doubao"

    def _call(self, prompt: str, **kwargs: Any) -> str:
        try:
            logging.info("Making API call to Doubao endpoint.")
            return self._make_api_call(prompt)
        except Exception as e:
            logging.error(f"Error in DoubaoLLM _call: {e}", exc_info=True)

    def _make_api_call(self, prompt: str) -> str:
        completion = self.client.chat.completions.create(
            model = self.model,
            messages = [
                {
                    "role": "user",
                    "content": prompt,
                },
            ]
        )
        return completion.choices[0].message.content


llmdoubao = DoubaoLLM(
    akstr="AKLTM2ZkYzczMThlMWM5NGYxYTg5NDE4N2ZlYmRhOTNlZDA", 
    skstr="TXpZME9EWTBOVGcwTTJVNU5EQmxPVGc0WkRkak16Rm1ORFJoTjJNd1pEQQ=="
)

prompt_template = "你是一个智能助手，回答以下的问题： {question}"
promptdoubao = PromptTemplate(
    input_variables=["question"], template=prompt_template
)
chat = LLMChain(llm=llmdoubao, prompt=promptdoubao)

chat("姚明身高是多少")

{'question': '姚明身高是多少',
 'text': '姚明的身高是 2.26 米。\n\n姚明是中国著名篮球运动员，他在篮球领域取得了卓越的成就。'}